In [23]:
#creating LLm nano GPT 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import tiktoken

In [2]:
#downloading Dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-10 16:17:25--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.89MB/s    in 0.4s    

2023-12-10 16:17:26 (2.89 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# reading input file just to check
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)
print(f"Vocab size: {vocab_size}")
print(f"Unique chars: {''.join(unique_chars)}")

Vocab size: 65
Unique chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [10]:
#string to intiger dic
stoi = { c:i for i, c in enumerate(unique_chars)}
itos = { i:c for i, c in enumerate(unique_chars)}

encoder = lambda s:[stoi[x] for x in s]
decoder = lambda l: ''.join([itos[i] for i in l])

In [18]:
# encoding the input shakespere text 

data = encoder(text)

torch_data = torch.tensor(data)
print(torch_data[:1000])
print(torch_data.shape)


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [20]:
enc = tiktoken.get_encoding('gpt2')


In [21]:
#enc input text
enc.encode(text)

[5962,
 22307,
 25,
 198,
 8421,
 356,
 5120,
 597,
 2252,
 11,
 3285,
 502,
 2740,
 13,
 198,
 198,
 3237,
 25,
 198,
 5248,
 461,
 11,
 2740,
 13,
 198,
 198,
 5962,
 22307,
 25,
 198,
 1639,
 389,
 477,
 12939,
 2138,
 284,
 4656,
 621,
 284,
 1145,
 680,
 30,
 198,
 198,
 3237,
 25,
 198,
 4965,
 5634,
 13,
 12939,
 13,
 198,
 198,
 5962,
 22307,
 25,
 198,
 5962,
 11,
 345,
 760,
 327,
 1872,
 385,
 1526,
 28599,
 318,
 4039,
 4472,
 284,
 262,
 661,
 13,
 198,
 198,
 3237,
 25,
 198,
 1135,
 760,
 470,
 11,
 356,
 760,
 470,
 13,
 198,
 198,
 5962,
 22307,
 25,
 198,
 5756,
 514,
 1494,
 683,
 11,
 290,
 356,
 1183,
 423,
 11676,
 379,
 674,
 898,
 2756,
 13,
 198,
 3792,
 470,
 257,
 15593,
 30,
 198,
 198,
 3237,
 25,
 198,
 2949,
 517,
 3375,
 319,
 470,
 26,
 1309,
 340,
 307,
 1760,
 25,
 1497,
 11,
 1497,
 0,
 198,
 198,
 12211,
 22307,
 25,
 198,
 3198,
 1573,
 11,
 922,
 4290,
 13,
 198,
 198,
 5962,
 22307,
 25,
 198,
 1135,
 389,
 17830,
 3595,
 4290,
 11,
 262,
 1458,


In [22]:
#import sequential train test split
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(torch_data, test_size=0.1, shuffle=False)

In [ ]:
#batching data

block_size = 8
train_loader = DataLoader(train_data, batch_size=block_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=block_size, shuffle=False)